# COLAB SETUP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys, os

if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    ! git clone https://github.com/birshert/attack-lang-models

    %cd attack-lang-models
    
    ! pip install -r requirements.txt
    
    import nltk
    nltk.download('punkt')
    
    ! cp /content/drive/MyDrive/*.pth /content/attack-lang-models/models/

# MAIN BODY

In [ ]:
import model

In [ ]:
model_ = model.Model()
model_.load_models()
model_.to('cuda:0')

In [ ]:
model_.russian_forward()
model_.set_nonrussian_grad_zero()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

In [ ]:
import torch

a = tokenizer.encode("I went fishing and I caught several big fish", return_tensors='pt', max_length=30, padding='max_length').flatten()
b = tokenizer.encode("I went fishing and I <mask> several big fish", return_tensors='pt', max_length=30, padding='max_length').flatten()

c = torch.stack((a, a)).to('cuda:0')
d = torch.stack((b, b)).to('cuda:0')

In [ ]:
optimizer = torch.optim.Adam(model_.parameters())

In [ ]:
a = model_(c, d)

In [ ]:
a.backward()

In [ ]:
optimizer.step()

In [ ]:
model_.mlm_model.lm_head.decoder.weight

In [ ]:
logits = model_.mlm_model(tokenizer.encode("I went fishing and I <mask> several big fish", return_tensors='pt').to('cuda:0')).logits

In [ ]:
tokenizer.decode([torch.argmax(logits[:, torch.where(tokenizer.encode("I went fishing and I <mask> several big fish", return_tensors='pt') == tokenizer.mask_token_id)[1], :]).item()])